<a href="https://colab.research.google.com/github/sanyakapoor27/ensemble_segmentation/blob/main/Ensemble_semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install segmentation_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install kaggle

In [ ]:
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [ ]:
# importing libraries
import cv2
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
from keras.utils import normalize
from keras.metrics import MeanIoU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import load_model
from skimage.io import imread
from keras.layers import Input, Conv2D
from keras.models import Model
import random

Segmentation Models: using `tf.keras` framework.


In [ ]:
#setting gpu memory consumption growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#downloading kaggle.json file
!gdown "https://drive.google.com/uc?id=1UvRuiTaifBWLHnueH8z4Bj5gihcaxmiM"

Downloading...
From: https://drive.google.com/uc?id=1UvRuiTaifBWLHnueH8z4Bj5gihcaxmiM
To: /content/kaggle.json
100% 70.0/70.0 [00:00<00:00, 54.3kB/s]


In [ ]:
#making kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# downloading the dataset from kaggle
!kaggle datasets download -d dat0chin/electron-microscopy-dataset

 88% 190M/215M [00:00<00:00, 216MB/s]
100% 215M/215M [00:01<00:00, 207MB/s]


In [ ]:
!unzip /content/electron-microscopy-dataset.zip

Archive:  /content/electron-microscopy-dataset.zip
  inflating: test/data/test0.tif     
  inflating: test/data/test1.tif     
  inflating: test/data/test10.tif    
  inflating: test/data/test100.tif   
  inflating: test/data/test101.tif   
  inflating: test/data/test102.tif   
  inflating: test/data/test103.tif   
  inflating: test/data/test104.tif   
  inflating: test/data/test105.tif   
  inflating: test/data/test106.tif   
  inflating: test/data/test107.tif   
  inflating: test/data/test108.tif   
  inflating: test/data/test109.tif   
  inflating: test/data/test11.tif    
  inflating: test/data/test110.tif   
  inflating: test/data/test111.tif   
  inflating: test/data/test112.tif   
  inflating: test/data/test113.tif   
  inflating: test/data/test114.tif   
  inflating: test/data/test115.tif   
  inflating: test/data/test116.tif   
  inflating: test/data/test117.tif   
  inflating: test/data/test118.tif   
  inflating: test/data/test119.tif   
  inflating: test/data/test12.tif    

In [ ]:
def preprocessing(dir_name, img_size):

  preprocessed_image = []

  for image in os.listdir(dir_name):
    if image.lower().endswith('.tif'):
      path = os.path.join(dir_name, image)

      image = imread(path)
      image = cv2.resize(image, img_size)
      #image = image / 255.
      preprocessed_image.append(image)

  preprocessed_image = np.array(preprocessed_image)

  print("Preprocessing is complete!")
  return preprocessed_image

In [ ]:
train_dir = "/content/train/data"
train_maskdir = "/content/train/label"
test_dir = "/content/test/data"
test_maskdir = "/content/train/label"

In [ ]:
train_images = preprocessing(train_dir, (256,256))
train_masks = preprocessing(train_maskdir, (256,256))
test_images = preprocessing(test_dir, (256,256))
test_masks = preprocessing(test_maskdir, (256,256))

Preprocessing is complete!
Preprocessing is complete!
Preprocessing is complete!
Preprocessing is complete!


In [ ]:
re_train_images = np.expand_dims(train_images, axis=-1)
re_train_masks = np.expand_dims(train_masks, axis=-1)
re_test_images = np.expand_dims(test_images, axis=-1)
re_test_masks = np.expand_dims(test_masks, axis=-1)

In [ ]:
re_train_images.shape

(165, 256, 256, 1)

In [ ]:
# Check unique values in the expanded masks
unique_values = np.unique(re_test_images)

print("Unique Values in Images:", unique_values)

Unique Values in Images: [ 15  23  32  34  35  39  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 233
 234 235 236 237 238 239 241 244 246]


In [ ]:
def binary_mask(mask):
  threshold = 128
  binary_masks = (mask >= threshold).astype(np.uint8)
  return binary_masks

In [ ]:
#parameters for our four models

total_loss = sm.losses.dice_loss
num_classes=1
activation='sigmoid'
LR = 0.0001
optim = keras.optimizers.Adam(LR)
metrics = [ sm.metrics.IOUScore(threshold=0.6), sm.metrics.FScore(threshold=0.6)]

In [ ]:
#model 1

BACKBONE1 = 'vgg16'

vgg1_model = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=1, activation=activation)

inp = Input(shape=(256, 256, 1))
l1 = Conv2D(3, (1, 1))(inp) # map data to 3 channels
out = vgg1_model(l1)

vgg1_model = Model(inp, out, name=vgg1_model.name)

#compiling the model
vgg1_model.compile(optim, total_loss, metrics=metrics)

print(vgg1_model.summary())

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 3)       6         
                                                                 
 model (Functional)          (None, None, None, 1)     23752273  
                                                                 
Total params: 23752279 (90.61 MB)
Trainable params: 23748247 (90.59 MB)
Non-trainable params: 4032 (15.75 KB)
_________________________________________________________________
None


In [ ]:
type(re_test_masks)

numpy.ndarray

In [ ]:
re_test_masks = re_test_masks.astype('float32')
re_train_masks = re_train_masks.astype('float32')

In [ ]:
vgg1_history = vgg1_model.fit(re_train_images,
          re_train_masks,
          batch_size=8,
          epochs=5,
          verbose=1,
          validation_data=(re_test_images, re_test_masks))

In [ ]:
vgg1_model.save('vgg1_backbone_unet.hdf5')

In [ ]:
#model 2

BACKBONE3 = 'vgg19'

vgg_model = sm.Unet(BACKBONE3, encoder_weights='imagenet', classes=num_classes, activation=activation, encoder_freeze=True)

inp = Input(shape=(256, 256, 1))
l1 = Conv2D(3, (1, 1))(inp) # map data to 3 channels
out = vgg_model(l1)

vgg_model = Model(inp, out, name=vgg_model.name)

#compiling the model
vgg_model.compile(optim, total_loss, metrics=metrics)

print(vgg_model.summary())

In [ ]:
vgg_history = vgg_model.fit(re_train_images,
          train_masks,
          batch_size=8,
          epochs=5,
          verbose=1,
          validation_data=(re_test_images, test_masks))

In [ ]:
vgg_model.save('vgg_backbone_unet.hdf5')

In [ ]:
model1 = load_model('/content/vgg1_backbone_unet.hdf5', compile=False)
model2 = load_model('/content/vgg_backbone_unet.hdf5', compile=False)

#Weighted average ensemble
models = [model1, model2]

pred1 = model1.predict(vgg1_test_images)
pred2 = model2.predict(vgg_test_images)

preds=np.array([pred1, pred2])

#preds=np.array(preds)
weights = [0.1, 0.9]

#Use tensordot to sum the products of all elements over specified axes.
weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
weighted_ensemble_prediction = np.argmax(weighted_preds, axis=3)

y_pred1_argmax=np.argmax(pred1, axis=3)
y_pred2_argmax=np.argmax(pred2, axis=3)

In [ ]:
#Using built in keras function
num_classes = 1
IOU1 = MeanIoU(num_classes)
IOU2 = MeanIoU(num_classes)

IOU_weighted = MeanIoU(num_classes)

IOU1.update_state(test_masks[:,:,:,0], y_pred1_argmax)
IOU2.update_state(test_masks[:,:,:,0], y_pred2_argmax)

IOU_weighted.update_state(test_masks[:,:,:,0], weighted_ensemble_prediction)


print('IOU Score for model1 = ', IOU1.result().numpy())
print('IOU Score for model2 = ', IOU2.result().numpy())
print('IOU Score for weighted average ensemble = ', IOU_weighted.result().numpy())

In [ ]:
#Grid search for the best combination of weights

import pandas as pd
df = pd.DataFrame([])

for w1 in range(0, 10):
    for w2 in range(0,10):
            wts = [w1/10.,w2/10.]

            IOU_wted = MeanIoU(num_classes)
            wted_preds = np.tensordot(preds, wts, axes=((0),(0)))
            wted_ensemble_pred = np.argmax(wted_preds, axis=3)
            IOU_wted.update_state(test_masks[:,:,:,0], wted_ensemble_pred)
            print("Now predciting for weights :", w1/10., w2/10., " : IOU = ", IOU_wted.result().numpy())
            df = df.append(pd.DataFrame({'wt1':wts[0],'wt2':wts[1],
                                         'IOU': IOU_wted.result().numpy()}, index=[0]), ignore_index=True)

max_iou_row = df.iloc[df['IOU'].idxmax()]
print("Max IOU of ", max_iou_row[2], " obtained with w1=", max_iou_row[0],
      " w2=", max_iou_row[1])

In [ ]:
opt_weights = [max_iou_row[0], max_iou_row[1]]

#Use tensordot to sum the products of all elements over specified axes.
opt_weighted_preds = np.tensordot(preds, opt_weights, axes=((0),(0)))
opt_weighted_ensemble_prediction = np.argmax(opt_weighted_preds, axis=3)

In [ ]:
#Predict on a few images

test_img_number = random.randint(0, len(test_images))
test_img = test_images[test_img_number]
ground_truth=test_masks[test_img_number]
test_img_norm=test_img[:,:,:]
test_img_input=np.expand_dims(test_img_norm, 0)

#Weighted average ensemble
models = [model1, model2]

test_img_input1 = preprocess_input1(test_img_input)
test_img_input2 = preprocess_input3(test_img_input)

test_pred1 = model1.predict(test_img_input1)
test_pred2 = model2.predict(test_img_input2)

test_preds=np.array([test_pred1, test_pred2])

#Use tensordot to sum the products of all elements over specified axes.
weighted_test_preds = np.tensordot(test_preds, weights, axes=((0),(0)))
weighted_ensemble_test_prediction = np.argmax(weighted_test_preds, axis=3)[0,:,:]

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(221)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')

plt.subplot(222)
plt.title('Prediction on test image')
plt.imshow(weighted_ensemble_test_prediction, cmap='jet')
plt.show()